# Program Project 1: Alignments DNA+Protein alignment

12/12/2025
Tiffany Lin

## Project Question
Align a DNA sequence and a protein sequence based on the translated sequences of the DNA sequences.
Only the **forward reading frames** need be considered. If desired, you may use existing packages for
reading/writing sequences, scoring tables, and formatting the results. Display the results in conventional
pairs of lines format, i.e., in blocks with the aligned sequences above each other, showing both the DNA
and translated protein sequences.

- Use a local alignment algorithm
  - Bonus: +10 pts. Implement both global and local alignments
- Assume the sequences are in fasta format
- Use a scoring table such as Blosum or PAM read in from a file. A good source of scoring matrix files is [ftp://ftp.ncbi.nlm.nih.gov/blast/matrices/](ftp://ftp.ncbi.nlm.nih.gov/blast/matrices/)
- Allow affine gaps within reading frames
- Allow length independent gaps between reading frames.
  - Bonus + 5 pts. : allow affine gaps between reading frames

## Introduction

DNA–protein alignment is used to identify coding regions and validating gene models. Because proteins are encoded by DNA codons, a DNA sequence must be translated into amino acids before it can be aligned to a protein sequence using standard substitution matrices. 

In this project, I aligned a DNA sequence to a protein sequence by translating the DNA into its three forward reading frames and performing local amino-acid alignment (Smith-Waterman) to identify the best-matching frame and region.

- Biopython [tutorial](https://biopython.org/docs/latest/Tutorial/chapter_introduction.html)

In [27]:
# Python version 3.12.4 on MacBook Pro (M4)
# pip install biopython

import Bio
print(Bio.__version__)

1.85


## Methods

- Input data: FASTA DNA and FASTA protein 
  1.  Synthetic test sequence. Create a 50 bp sequence as a test model
  2.  [NCBI Coding Sequence file (E.coli)](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/)
      1. For more information about Data Acquisition, please refer to file `cds_data.md` with additional background and instructions
- Translation: three forward frames
- Scoring: BLOSUM62 loaded from NCBI file
- Alignment: Smith–Waterman local alignment with **affine gaps (gap open/extend)**
- Frame selection: pick highest score
- Output formatting: map aligned AA back to DNA codons and print paired lines


### 1. Data Preparation + Translation to Forward Reading Frames

#### 1.1 Synthetic Test Sequence: Create a small data for testing (~50 bp) 
**(Comment out this part when reading the actual E.coli data)**

1. Create a ~45 bp test nucleotide sequence `dna`
2. Convert `dna` to a.a. **3 forward frames** (15*3 = 45 b.p.) using Biopython `.translate()` + Add STOP CODON
3. Save `.fa` file
4. Verify translation

https://biopython.org/docs/latest/Tutorial/chapter_seq_annot.html

In [28]:
 from Bio.Seq import Seq
 from Bio.SeqRecord import SeqRecord # for handling sequence records + metadata
 from Bio import SeqIO  # for input and output of sequence files reading
 import random
 import os
 
 # Create a short DNA sequence (45-48 bp) and translate it`
 # Pick 15 codons you like (3*15 = 45 nt). Add STOP (TAA) if you want ~48 bp total.
 codons = ["ATG","GCT","TGG","CTG","AAA","GAG","CAG","ATC","GTT","CAC","GGT","TCC","ACC","TAC","GCT"]
 dna = "".join(codons) + "TAA"   # add STOP bp
 prot = Seq(dna).translate(to_stop=False)  # -> 16 aa incl. * at end
 dna_seq = Seq(dna)
 
 
 # === translate full sequence ===
 protein_seq = dna_seq.translate(to_stop=True)
 # Wrap protein in a SeqRecord
 prot_rec = SeqRecord(protein_seq, id="test_protein", description="translated peptide (15 aa)")
 
 print("DNA len:", len(dna), "Protein:", protein_seq)
 
 # === translate 3 forward frames ===
 frames_aa = [dna_seq[i:].translate(to_stop=False) for i in range(3)]    # to_stop=False keeps '*' if a stop appears; fine for alignment/debug
 
 print("Frame 0:", frames_aa[0])
 print("Frame 1:", frames_aa[1])
 print("Frame 2:", frames_aa[2])
 
# ==== OPTIONAL: save FASTAs under pp1/data/ ====
# os.makedirs("pp1/data", exist_ok=True)
# SeqIO.write(SeqRecord(dna_seq, id="test_dna", description="~50bp"), "data/small_dna.fa", "fasta")
# SeqIO.write(SeqRecord(protein_seq, id="test_protein", description="translated peptide (15 aa)"), "data/small_protein.fa", "fasta")
# SeqIO.write(SeqRecord(frames_aa[0], id="rf0_aa", description="translated frame 0"), "data/rf0_aa.fa", "fasta")
# SeqIO.write(SeqRecord(frames_aa[1], id="rf1_aa", description="translated frame 1"), "data/rf1_aa.fa", "fasta")
# SeqIO.write(SeqRecord(frames_aa[2], id="rf2_aa", description="translated frame 2"), "data/rf2_aa.fa", "fasta")

DNA len: 48 Protein: MAWLKEQIVHGSTYA
Frame 0: MAWLKEQIVHGSTYA*
Frame 1: WLG*KSRSFTVPPTL
Frame 2: GLAERADRSRFHLRL


/opt/anaconda3/lib/python3.12/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


#### 1.2 RefSeq CDS and protein data (E. coli) 
**(Comment out this part when using the 50 bp synthetic test sequence)**

In [29]:
# from Bio import SeqIO
# from Bio.Seq import Seq
# from Bio.SeqRecord import SeqRecord
# 
# # === Read CDS and protein from FASTA files ===
# cds_rec = SeqIO.read("/Users/rusher/Desktop/biol595_extra_credit/pp1/data/real_cds.fa", "fasta")
# prot_rec = SeqIO.read("/Users/rusher/Desktop/biol595_extra_credit/pp1/data/real_protein.fa", "fasta")
# 
# dna_seq = cds_rec.seq
# protein_seq = prot_rec.seq
# 
# print("CDS ID:", cds_rec.id)
# print("CDS length:", len(dna_seq), "mod 3:", len(dna_seq) % 3) # check length mod 3
# print("Protein ID:", prot_rec.id)
# print("Protein length:", len(protein_seq))
# 
# # === Translate 3 forward reading frames from CDS ===
# def translate_frame(dna: Seq, frame: int):
#     trimmed_len = ((len(dna) - frame) // 3) * 3
#     return dna[frame:frame+trimmed_len].translate(to_stop=False)
# 
# frames_aa = [translate_frame(dna_seq, f) for f in range(3)]
# 
# print("Frame 0:", frames_aa[0])
# print("Frame 1:", frames_aa[1])
# print("Frame 2:", frames_aa[2])


### 2. Substitution Scoring Matrix: Load BLOSUM62 from a file

After getting the DNA sequence and translating it into the three possible forward reading frames, the next requirement is to define **how amino acids are scored during alignment**.

Protein–protein alignment algorithms such as Smith–Waterman do not simply check whether two amino acids are identical. Instead, they rely on a **substitution scoring matrix** that reflects the biological likelihood of one amino acid being replaced by another during evolution. For example, substituting tryptophan with phenylalanine is more biologically plausible than substituting tryptophan with glycine.

To get this biological information, we use the **BLOSUM62** amino-acid substitution matrix. BLOSUM62 assigns log-odds scores to all possible amino-acid pairs based on observed substitution frequencies in conserved protein blocks. **Positive scores correspond to commonly observed or conservative substitutions, while negative scores penalize unlikely substitutions.**

In the following code cell, the BLOSUM62 matrix is read from an [**NCBI BLAST-format file**](https://ftp.ncbi.nlm.nih.gov/blast/matrices/BLOSUM62) and **converted into a dictionary** that can be used by the local alignment algorithm.



In [ ]:
# function to read BLOSUM matrix from NCBI format
def read_blosum_ncbi(path: str):
    """
    Reads an amino-acid substitution matrix in NCBI BLAST format
    (such as BLOSUM62) and converts it into a dictionary suitable
    for pairwise sequence alignment.

    The output dictionary maps amino-acid pairs (rowAA, colAA)
    to their integer substitution score.
    """
    mat = {}
    with open(path, "r") as f:
        # keep only non-empty, non-comment lines
        lines = [ln.strip() for ln in f if ln.strip() and not ln.startswith("#")]

    # The first remaining line is the column header listing amino acids
    # Example: A R N D C Q E G H I L K M F P S T W Y V B Z X *
    header = lines[0].split()  
    for ln in lines[1:]:
        parts = ln.split()
        row = parts[0]
        scores = parts[1:]

        # Store score for each (row, column) amino-acid pair
        for col, sc in zip(header, scores):
            mat[(row, col)] = int(sc)
    return mat

# Store score for each (row, column) amino-acid pair
matrix = read_blosum_ncbi("/Users/rusher/Desktop/biol595_extra_credit/pp1/data/BLOSUM62.txt")

In [31]:
print(matrix[('A','H')])  # -2
print(matrix[('W','W')])  # 11
print(matrix[('*','A')])  # -4
print(matrix[('*','*')])  # 1

-2
11
-4
1


### 3. Local DNA-Protein Alignment

After defining the amino-acid substitution scoring scheme (BLOSUM62), the next step is to perform
sequence alignment. The DNA sequence was translated into its three possible forward reading frames,
each producing a different amino-acid sequence. Because the correct reading frame is unknown,
the protein sequence is locally aligned against **each translated frame independently**.

Local alignment (Smith–Waterman) is used because the protein may correspond to only a subsequence
of the translated DNA. Alignments are scored using the BLOSUM62 substitution matrix together with
**affine gap penalties**, where opening a gap incurs a larger penalty than extending an existing gap.
This models biologically realistic insertions and deletions within a reading frame.

For each reading frame, the best-scoring local alignment is retained. The reading frame that yields
the highest alignment score is selected as the most likely frame corresponding to the protein.

#### 3.1 Frame-wise Smith Waterman alignment

In [ ]:
from Bio import pairwise2

# Gap penalties for affine gap model
gap_open = 10
gap_extend = 1

best_by_frame = []

# Perform local alignment for each of the three forward reading frames
for f in range(3):
    rf = str(frames_aa[f])       # translated AA from DNA frame f
    prot = str(protein_seq)      # target protein AA sequence
    
    # Perform local Smith–Waterman alignment using:
    # - BLOSUM62 substitution matrix
    # - affine gap penalties (open and extend)
    alns = pairwise2.align.localds(rf, prot, matrix, -gap_open, -gap_extend)

    # Select the highest-scoring alignment for this frame
    # Alignment tuple format:
    # (rf_aln, prot_aln, score, begin, end)
    best = max(alns, key=lambda a: a.score)  
    best_by_frame.append(best)
    
    print(f"Frame {f} best local score = {best.score}")

Frame 0 best local score = 81.0
Frame 1 best local score = 15.0
Frame 2 best local score = 9.0


In [ ]:
# Identify the reading frame with the highest alignment score
# This frame is interpreted as the most likely coding frame
best_frame = max(range(3), key=lambda f: best_by_frame[f].score)
rf_aln, prot_aln, score, begin, end = best_by_frame[best_frame]

print("\nBest frame:", best_frame, "score:", score)

# Display the alignment in a conventional paired-lines format
# showing the aligned amino-acid sequences and match indicators
print(pairwise2.format_alignment(rf_aln, prot_aln, score, begin, end))


Best frame: 0 score: 81.0
1 MAWLKEQIVHGSTYA
  |||||||||||||||
1 MAWLKEQIVHGSTYA
  Score=81



#### 3.2 Paired line output

To display the alignment in paired-lines format, the translated amino-acid alignment was mapped back to the original DNA sequence at the codon level. 

First, the DNA sequence was split into codons according to the selected reading frame. Then, the amino-acid alignment was traversed position by position: aligned amino acids were assigned their corresponding DNA codons, **while gap characters `(-)` in the protein alignment were represented as triple-base gaps `(---)` in the DNA line.** This approach preserves alignment structure while maintaining the correct correspondence between amino acids and their encoding codons.

In [ ]:
# converts a DNA string into a list of codons for a specific reading frame.
def codons_for_frame(dna_str: str, frame: int, drop_terminal_stop=True):
    """
    Split a DNA sequence into codons for a given reading frame.

    Parameters:
    - dna_str: DNA sequence as a string
    - frame: reading frame (0, 1, or 2)
    - drop_terminal_stop: whether to remove a terminal stop codon

    Returns:
    - List of codon strings (length 3 each)
    """
    trimmed_len = ((len(dna_str) - frame) // 3) * 3
    codons = [dna_str[i:i+3] for i in range(frame, frame + trimmed_len, 3)]
    if drop_terminal_stop and codons and codons[-1] in {"TAA", "TAG", "TGA"}: # remove terminal stop codon
        codons = codons[:-1]
    return codons

# Map aligned amino acids back to their corresponding DNA codons
def codons_under_alignment(dna_str: str, frame: int, rf_aln: str):
    """
    Generate a codon line that matches an amino-acid alignment.

    Gaps in the amino-acid alignment are represented as '---'
    in the DNA codon line to preserve alignment structure.
    """
    codons = codons_for_frame(dna_str, frame, drop_terminal_stop=False)
    j = 0 # index for codons
    out = []
    for aa in rf_aln:
        if aa == '-':
            out.append('---')
        else:
            out.append(codons[j] if j < len(codons) else '---')
            j += 1
    return out

dna_str = str(dna_seq)
codon_line = codons_under_alignment(dna_str, best_frame, rf_aln)

print("Paired-lines display (Protein + DNA codons):")
print(prot_aln)
print(" ".join(codon_line))

Paired-lines display (Protein + DNA codons):
MAWLKEQIVHGSTYA-
ATG GCT TGG CTG AAA GAG CAG ATC GTT CAC GGT TCC ACC TAC GCT TAA


Terminal stop codons do not encode amino acids and therefore appear as gap characters (- in the protein alignment and --- in the DNA codon line) in the paired-lines display.


### 4. Validation: Frameshift Test

In the matched CDS + protein case, frame 0 is expected to produce the best alignment because the CDS is already in the correct coding frame. To verify that the frame-selection procedure behaves correctly when the DNA start position is offset, I repeated the analysis after **shifting the CDS by 1 nucleotide and 2 nucleotides**. A +1 shift should move the correct coding frame to frame 2, and a +2 shift should move it to frame 1 (relative to the shifted sequence start).

`best_local_aligment_over_frames`: Although frame-by-frame alignment was already performed earlier, this function groups the same steps into a single reusable procedure. This makes it easy to repeat the analysis on frame-shifted DNA sequences and to show that the correct reading frame is chosen based on alignment scores, not because the frame was assumed in advance.

In [36]:
def best_local_alignment_over_frames(dna: Seq, protein_seq: Seq, matrix, gap_open=10, gap_extend=1):
    """
    Translate the input DNA into the three forward reading frames (0/1/2),
    locally align each translated frame to the target protein sequence,
    and return:
      1) the best alignment from each frame
      2) the index of the frame with the highest alignment score
    """
    frames = [translate_frame(dna, f) for f in range(3)]
    
    best_by_frame = []
    for f in range(3):
        rf = str(frames[f])
        prot = str(protein_seq)
        alns = pairwise2.align.localds(rf, prot, matrix, -gap_open, -gap_extend)
        best = max(alns, key=lambda a: a.score)
        best_by_frame.append(best)
    
    best_frame = max(range(3), key=lambda f: best_by_frame[f].score)
    return best_by_frame, best_frame

In [37]:
#==== Validation: frame-offset stress test ====

# Original (should be frame 0 for matched CDS+protein)
best0, bf0 = best_local_alignment_over_frames(dna_seq, protein_seq, matrix, gap_open, gap_extend)

# Create artificial frame shifts by removing 1 or 2 nucleotides from the front
dna_shift1 = dna_seq[1:]
dna_shift2 = dna_seq[2:]

# Re-run frame selection on shifted sequences
best1, bf1 = best_local_alignment_over_frames(dna_shift1, protein_seq, matrix, gap_open, gap_extend)
best2, bf2 = best_local_alignment_over_frames(dna_shift2, protein_seq, matrix, gap_open, gap_extend)

print("Original DNA best-frame scores:", [b.score for b in best0], "=> best frame", bf0)
print("Shift +1 nt best-frame scores:", [b.score for b in best1], "=> best frame", bf1)
print("Shift +2 nt best-frame scores:", [b.score for b in best2], "=> best frame", bf2)

Original DNA best-frame scores: [81.0, 15.0, 9.0] => best frame 0
Shift +1 nt best-frame scores: [15.0, 9.0, 76.0] => best frame 2
Shift +2 nt best-frame scores: [9.0, 76.0, 6.0] => best frame 1


## Results

### Synthetic test sequence

For the synthetic ~50 bp test sequence, the DNA was translated into three forward reading frames and locally aligned to the corresponding protein sequence using the BLOSUM62 substitution matrix with affine gap penalties. As expected, frame 0 produced the highest local alignment score (81.0) and recovered the full protein sequence with no gaps, while frames 1 and 2 showed substantially lower scores (15.0 and 9.0) due to premature stop codons and mismatches. This confirmed that the alignment and frame-selection logic functions correctly on a controlled example.

### RefSeq CDS–protein alignment (E. coli)

Using a curated CDS–protein pair from *E. coli* sequence, the three translated forward frames were independently aligned to the annotated protein sequence. Frame 0 yielded the highest local alignment score (103.0), while frames 1 and 2 produced much lower scores (11.0 and 6.0, respectively). The best alignment for frame 0 matched the full-length protein sequence exactly.

The paired-lines output correctly displayed the aligned protein sequence above the corresponding DNA codons, demonstrating that the alignment results can be mapped back to nucleotide space in a biologically interpretable format.

### Frame-offset validation

To further validate frame detection, the CDS sequence was artificially shifted by one and two nucleotides prior to translation. After a +1 nucleotide shift, the best-scoring alignment moved from frame 0 to frame 2, and after a +2 nucleotide shift, the best alignment moved to frame 1. This behavior matches the expected relationship between nucleotide offsets and reading-frame numbering, confirming that the algorithm reliably identifies the correct coding frame based on alignment score.

## Discussion and Conclusion

This project demonstrates a complete pipeline for aligning a DNA sequence to a protein sequence using translation-based alignment and forward reading frame detection. By translating the DNA into its three forward reading frames and performing local Smith–Waterman alignment with BLOSUM62 matrix, the correct coding frame can be identified based on alignment score alone.

Both synthetic and real biological test cases show that the true coding frame consistently produces the highest alignment score, while alternative frames are penalized by stop codons and biologically implausible substitutions. The paired lines output further confirms that a.a. alignments can be mapped back to the DNA codons, satisfying the requirement to display both protein and nucleotide sequences together. Affine gap penalties were also used within reading frames to model biologically realistic insertions and deletions. 

Overall, this project successfully implied translation-based DNA–protein alignment using local Smith–Waterman alignment, affine gap penalties, and curated biological data, and reliably identifies the correct forward reading frame across artificial and biological test cases.

## Reference
> Cock, P. J., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., Friedberg, I., Hamelryck, T., Kauff, F., Wilczynski, B., & de Hoon, M. J. (2009). Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics (Oxford, England), 25(11), 1422–1423. https://doi.org/10.1093/bioinformatics/btp163
> 
> NCBI RefSeq database: https://www.ncbi.nlm.nih.gov/refseq/